# Prepocesamiento de texto
En este Notebook se presenta el procesamiento de informacion realizado para implementar el proyecto "Mex-IA: Chatbot basado en RAG enfocado al apoyo legal del ciudadano" sometido al concurso "Acelerando Mexico con Inteligencia Articfial", organizado por Intel.

El objetivo de este Notebook es identificar y extraer la informacion de cada uno de los articulos incluidos en el documento oficial de la constitucion Mexicana. Para lograr dicho objetivo, se optop por usar Expresiones Regulares (RE, Regular Expressions) para identificar el inicio y final de cada aritculo. Posteriormente, Se utiliza la libreria Llama Index para transformar el contenido de cada articulo en una representacion numerica atrves de la evaluacion del texto en Embedings.

El contenido del Notebook se divide como sigue:
1. Extraccion del contenido de articulos
2. Transformacion a representacion numerica
3. Validacion de la lectura de articulos en su representacion numerica

## 1. Extracción del contenido de artículos
A continuacion se implementa una funcion para procesar el contenido de cada pagina del documento de la constitucion en pdf. El objetivo de esta funcion es eliminar informacion redundante, como lo son el pie de pagina.

Con esta funcion se lee cada pagina del archivo y se extrae la informacion usando expresiones regulares para detectar el principio y fin de cada articulo. Una vez extraida la informacion se almacena el contenido de cada articulo en archivos .txt, los cuales se encuentran en la carpeta [articulos](articulos).

In [1]:
# cargamos librerias
from PyPDF2 import PdfReader
import re

In [31]:
# funcion para eliminar el contenido inecesario del pie de cada pagina
def clean_text(page, split_sent=None):
    text = page.extract_text()
    splits = re.split(r"(\d+\sde\s382)", text)
    out = splits[-1]

    if split_sent != None:
        aux_split = re.split(f'{split_sent}', out)
        out = aux_split[0]

    return out.strip()

# extraccion del contenido de los articulos (no transitorios)
reader = PdfReader('data/CPEUM.pdf')
texto_completo = ''

# Extraer el texto completo del PDF
for pi in range(159):
    if pi==158:
        texto_completo += clean_text(page=reader.pages[pi], split_sent='Artículos Transitorios')
    else:
        texto_completo += clean_text(page=reader.pages[pi])

articulos = re.split(r"(Artículo\s+\d+o?.)", texto_completo)

# creamos archivos con el contenido del articulo
for i in range(1, len(articulos), 2):
    # El índice i contiene "Artículo X" y el índice i+1 contiene el contenido del artículo
    titulo_articulo = articulos[i].strip()
    print(titulo_articulo)
    contenido_articulo = f'{articulos[i].strip()}\n'+articulos[i + 1].strip()
    
    # Crear un documento con el artículo
    with open(f'articulos/{titulo_articulo.replace(".", "")}.txt', 'w') as f:
        f.write(contenido_articulo)

## 2. Transformacion a representacion numerica
Posteriormente se transforma la informacion extraida de cada articulo a una reresentacion numerica, ya que la tecnica RAG usada en la implementacion del modelo propuesto requiere este formato.

En primer lugar se define la configuracion a usar, incluyendo el modelo de lenguaje (`gpt-4o-mini`) y el tipo de embeding (`text-embedding-3-small`). Note que esta etapa del proyecto requiere de un API key de OPEN AI para usar los modelos preentrenados, para mas informacion consulte [link](link...). Tambien es necesario crear una sesion de Chroma DB para alamcenar el resultado de la transformacion numerica. Este paso es fundamental, ya que la evaluacion de la base de datos consume recursos del API, por lo cual es importante alamcenar el resultado con el fin de evitar costos incesesarios cada vez que se requiera usar el modelo.

In [1]:
# se configura el modelo de lenguaje y embedding
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.openai import OpenAIEmbedding
import os

# api key
key = ''
with open('openai_key.txt', 'r') as t:
    key = t.read()
os.environ["OPENAI_API_KEY"] = key

# configuracion del modelo y embeding
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.embed_model = embed_model
Settings.llm = OpenAI(model="gpt-4o-mini")

# creamos sesion para guardar Vector Store Index
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

#  se inicializa cliente y coleccion
db = chromadb.PersistentClient(path="./index_data")
chroma_collection = db.get_or_create_collection("quickstart")

# asignamos ubicacion del vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Con la configuracion definida previmante, se transforma el contenido de cada articulo en una representacion numerica y se alamcena el resultado en la carpeta [index_data](index_data).

In [2]:
from llama_index.core import Document

# Cargamos los documentos de los articulos
import os
data_path = 'articulos'
files = os.listdir(data_path)
documents = [] # lista de documentos

for fi in files:
    with open(f'{data_path}/{fi}', 'r') as t:
        auxd = Document(text=t.read(), metadata={"nombre":f"{fi.split('.')[0]}"})
        documents.append(auxd)

# se crean los index de los documentos y se almacenan
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

## 3. Validacion de la lectura de articulos en su representacion numerica
Por ultimo, se muestra el proceso necesario para recuperar la representacion numerica del contenido de cada articulo:

In [1]:
# para volver a cargar los index
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.openai import OpenAIEmbedding
import os
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# api key
key = ''
with open('openai_key.txt', 'r') as t:
    key = t.read()
os.environ["OPENAI_API_KEY"] = key

# configuracion del modelo y embeding
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.embed_model = embed_model
Settings.llm = OpenAI(model="gpt-4o-mini")


# inicia el cliente
db = chromadb.PersistentClient(path="./index_data")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# lee el index del contenido almacenado
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

Finalmente se muestra un ejemplo para verificar que el contenido de cada articulo se almaceno correctamente.

In [4]:
# ejemplo usando metadata
import pprint
from llama_index.core.retrievers import VectorIndexRetriever

# RAG para las top 2 coincidencias mas similares
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# pregunta original
query = "los mexicanos tienen derecho a la educacion?"
nodes = retriever.retrieve(query)

# accede a la informacion del retrieve dado el query
docs = ''
for ni in nodes:
    aux_dic = dict(ni)
    print(aux_dic)
    docs += f"Nombre del documento: {aux_dic['node'].metadata['nombre']}\Contenido: {aux_dic['node'].text} \n"
pprint.pprint(docs)